In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
240,DECEMBER,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
241,DECEMBER,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[224]
242,DECEMBER,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[225]
243,DECEMBER,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[226]


In [9]:
df_2019 = df[['Title']]

In [10]:
df_2019

,Title
0,Escape Room
1,Rust Creek
2,American Hangman
3,A Dog's Way Home
4,The Upside
...,...
240,Spies in Disguise
241,Little Women
242,1917
243,Just Mercy


In [11]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [13]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [14]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return int(result[0].id)

In [15]:
df_2019['id'] = df_2019['Title'].map(lambda x: ids(str(x)))

<ipython-input-15-94292c00d2b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['id'] = df_2019['Title'].map(lambda x: ids(str(x)))


In [16]:
df_2019

,Title,id
0,Escape Room,522681
1,Rust Creek,561362
2,American Hangman,567738
3,A Dog's Way Home,508763
4,The Upside,440472
...,...,...
240,Spies in Disguise,431693
241,Little Women,331482
242,1917,530915
243,Just Mercy,522212


In [17]:
df_2019[df_2019['id'].isnull()]

,Title,id


In [18]:
df_2019['id'] = df_2019['id'].astype('int')

<ipython-input-18-a9ad4c5cdd5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['id'] = df_2019['id'].astype('int')


In [19]:
df_2019

,Title,id
0,Escape Room,522681
1,Rust Creek,561362
2,American Hangman,567738
3,A Dog's Way Home,508763
4,The Upside,440472
...,...,...
240,Spies in Disguise,431693
241,Little Women,331482
242,1917,530915
243,Just Mercy,522212


In [20]:
def description(movie_id):
        description =""
    
        movie_details = tmdb_movie.details(movie_id)
        if movie_details['overview'] and movie_details['tagline']:
          return movie_details['overview'] + "@#$" + movie_details['tagline']
        elif movie_details['overview'] and not movie_details['tagline']:
          return movie_details['overview'] + "@#$" + ""
        elif not movie_details['overview'] and movie_details['tagline']:
          return ""+ "@#$" + movie_details['tagline']
        else:
          return "@#$"

In [21]:
df_2019['description'] = df_2019['id'].map(lambda x: description(int(x)))

<ipython-input-21-b1011f2bdffe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['description'] = df_2019['id'].map(lambda x: description(int(x)))


In [22]:
df_2019

,Title,id,description
0,Escape Room,522681,Six strangers find themselves in circumstances...
1,Rust Creek,561362,When an overachieving college senior makes a w...
2,American Hangman,567738,An unidentified man posts a live feed on socia...
3,A Dog's Way Home,508763,"The adventure of Bella, a dog who embarks on a..."
4,The Upside,440472,Phillip is a wealthy quadriplegic who needs a ...
...,...,...,...
240,Spies in Disguise,431693,Super spy Lance Sterling and scientist Walter ...
241,Little Women,331482,Four sisters come of age in America in the aft...
242,1917,530915,"At the height of the First World War, two youn..."
243,Just Mercy,522212,The powerful true story of Harvard-educated la...


In [23]:
df_2019['description'] = df_2019['description'].apply(lambda x: x.split("@#$"))

<ipython-input-23-3ea3c17b40b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['description'] = df_2019['description'].apply(lambda x: x.split("@#$"))


In [24]:
df_2019

,Title,id,description
0,Escape Room,522681,[Six strangers find themselves in circumstance...
1,Rust Creek,561362,[When an overachieving college senior makes a ...
2,American Hangman,567738,[An unidentified man posts a live feed on soci...
3,A Dog's Way Home,508763,"[The adventure of Bella, a dog who embarks on ..."
4,The Upside,440472,[Phillip is a wealthy quadriplegic who needs a...
...,...,...,...
240,Spies in Disguise,431693,[Super spy Lance Sterling and scientist Walter...
241,Little Women,331482,[Four sisters come of age in America in the af...
242,1917,530915,"[At the height of the First World War, two you..."
243,Just Mercy,522212,[The powerful true story of Harvard-educated l...


In [25]:
df_2019['overview'] = df_2019['description'].map(lambda x: x[0])
df_2019['tagline'] = df_2019['description'].map(lambda x: x[1])

<ipython-input-25-808d7c67d85e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['overview'] = df_2019['description'].map(lambda x: x[0])
<ipython-input-25-808d7c67d85e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['tagline'] = df_2019['description'].map(lambda x: x[1])


In [26]:
df_2019

,Title,id,description,overview,tagline
0,Escape Room,522681,[Six strangers find themselves in circumstance...,Six strangers find themselves in circumstances...,Find The Clues Or Die
1,Rust Creek,561362,[When an overachieving college senior makes a ...,When an overachieving college senior makes a w...,Choose your prey carefully
2,American Hangman,567738,[An unidentified man posts a live feed on soci...,An unidentified man posts a live feed on socia...,A new kind of trial
3,A Dog's Way Home,508763,"[The adventure of Bella, a dog who embarks on ...","The adventure of Bella, a dog who embarks on a...",A lot can happen between lost and found.
4,The Upside,440472,[Phillip is a wealthy quadriplegic who needs a...,Phillip is a wealthy quadriplegic who needs a ...,Based on a true story.
...,...,...,...,...,...
240,Spies in Disguise,431693,[Super spy Lance Sterling and scientist Walter...,Super spy Lance Sterling and scientist Walter ...,Super Spy. Super Fly.
241,Little Women,331482,[Four sisters come of age in America in the af...,Four sisters come of age in America in the aft...,Own your story
242,1917,530915,"[At the height of the First World War, two you...","At the height of the First World War, two youn...",Time is the enemy
243,Just Mercy,522212,[The powerful true story of Harvard-educated l...,The powerful true story of Harvard-educated la...,Every generation has its hero. Meet ours.


In [27]:
df_2019['comb'] = df_2019['overview'] + df_2019['tagline']
df_2019['comb'] = df_2019['comb'].fillna('')

In [28]:
df_2019 = df_2019.rename(columns={'Title':'title'})

In [29]:
df_2019 = df_2019.loc[:,['id','overview','tagline','title','comb']]

In [30]:
df_2019

,id,overview,tagline,title,comb
0,522681,Six strangers find themselves in circumstances...,Find The Clues Or Die,Escape Room,Six strangers find themselves in circumstances...
1,561362,When an overachieving college senior makes a w...,Choose your prey carefully,Rust Creek,When an overachieving college senior makes a w...
2,567738,An unidentified man posts a live feed on socia...,A new kind of trial,American Hangman,An unidentified man posts a live feed on socia...
3,508763,"The adventure of Bella, a dog who embarks on a...",A lot can happen between lost and found.,A Dog's Way Home,"The adventure of Bella, a dog who embarks on a..."
4,440472,Phillip is a wealthy quadriplegic who needs a ...,Based on a true story.,The Upside,Phillip is a wealthy quadriplegic who needs a ...
...,...,...,...,...,...
240,431693,Super spy Lance Sterling and scientist Walter ...,Super Spy. Super Fly.,Spies in Disguise,Super spy Lance Sterling and scientist Walter ...
241,331482,Four sisters come of age in America in the aft...,Own your story,Little Women,Four sisters come of age in America in the aft...
242,530915,"At the height of the First World War, two youn...",Time is the enemy,1917,"At the height of the First World War, two youn..."
243,522212,The powerful true story of Harvard-educated la...,Every generation has its hero. Meet ours.,Just Mercy,The powerful true story of Harvard-educated la...


In [31]:
df_2019.to_csv('desc2019.csv',index=False)

In [32]:
from google.colab import files
files.download("desc2019.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>